# Testing Stuff related to Volatility

In [1]:
from src import data_fetcher
import pandas as pd

In [2]:
data : pd.MultiIndex = data_fetcher.fetch_ohlcv(["NVDA", "AAPL"], "2024-01-01", "2024-12-31")
data.to_excel("NVDA_AAPL_ohlcv.xlsx")
data.head()

[*********************100%***********************]  2 of 2 completed



,Price,Open,High,Low,Close,Adj Close,Volume
Date,Ticker,,,,,,
2024-01-02,NVDA,49.243999,49.294998,47.595001,48.167999,48.146572,411254000
2024-01-03,NVDA,47.485001,48.183998,47.320000,47.569000,47.547840,320896000
2024-01-04,NVDA,47.766998,48.500000,47.507999,47.998001,47.976650,306535000
2024-01-05,NVDA,48.462002,49.547001,48.306000,49.097000,49.075153,415039000
2024-01-08,NVDA,49.512001,52.275002,49.479000,52.252998,52.229752,642510000


In [3]:
data['Adj Close'].head()

Date        Ticker
2024-01-02  NVDA      48.146572
2024-01-03  NVDA      47.547840
2024-01-04  NVDA      47.976650
2024-01-05  NVDA      49.075153
2024-01-08  NVDA      52.229752
Name: Adj Close, dtype: float64

In [4]:
daily_return_pct = data['Adj Close'].pct_change()
daily_return_pct

Date        Ticker
2024-01-02  NVDA           NaN
2024-01-03  NVDA     -0.012436
2024-01-04  NVDA      0.009018
2024-01-05  NVDA      0.022897
2024-01-08  NVDA      0.064281
                        ...   
2024-12-23  AAPL      0.003065
2024-12-24  AAPL      0.011478
2024-12-26  AAPL      0.003176
2024-12-27  AAPL     -0.013242
2024-12-30  AAPL     -0.013263
Name: Adj Close, Length: 502, dtype: float64

In [5]:
import numpy as np
daily_return_log = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
daily_return_log

Date        Ticker
2024-01-02  NVDA           NaN
2024-01-03  NVDA     -0.012514
2024-01-04  NVDA      0.008978
2024-01-05  NVDA      0.022638
2024-01-08  NVDA      0.062299
                        ...   
2024-12-23  AAPL      0.003060
2024-12-24  AAPL      0.011413
2024-12-26  AAPL      0.003171
2024-12-27  AAPL     -0.013331
2024-12-30  AAPL     -0.013352
Name: Adj Close, Length: 502, dtype: float64

### DataFlow Example

In [6]:
from src.volatility import volatility_summary

In [7]:
tickers = ['AAPL', 'MSFT', 'NVDA']
ohlcv_data = data_fetcher.fetch_ohlcv(tickers, "2024-01-01", "2024-12-31")
sector_map = data_fetcher.get_sector_mapping(tickers)
benchmark_df = data_fetcher.fetch_index_data("^GSPC", "2024-01-01", "2024-12-31")

[*********************100%***********************]  3 of 3 completed

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
print("sector_map:")
print(sector_map)
print(f"\nType: {type(sector_map)}")
print(f"Shape: {sector_map.shape}")
print(f"Columns: {sector_map.columns.tolist()}")
print(f"Index: {sector_map.index}")
print(f"Data types: {sector_map.dtypes}")

sector_map:
  Ticker      Sector
0   AAPL  Technology
1   MSFT  Technology
2   NVDA  Technology

Type: <class 'pandas.core.frame.DataFrame'>
Shape: (3, 2)
Columns: ['Ticker', 'Sector']
Index: RangeIndex(start=0, stop=3, step=1)
Data types: Ticker    object
Sector    object
dtype: object


In [9]:
# Let's check the ohlcv_data structure and test historical_volatility directly
print("ohlcv_data structure:")
print(f"Type: {type(ohlcv_data)}")
print(f"Index: {ohlcv_data.index}")
print(f"Columns: {ohlcv_data.columns}")
print("\nFirst few rows:")
print(ohlcv_data.head())

# Test historical volatility function directly
from src.volatility import historical_volatility
hist_vol = historical_volatility(ohlcv_data, window=20)
print(f"\nhistorical_volatility output:")
print(f"Type: {type(hist_vol)}")
print(f"Shape: {hist_vol.shape}")
print(f"Index: {hist_vol.index}")
print("Sample values:")
print(hist_vol.tail())

ohlcv_data structure:
Type: <class 'pandas.core.frame.DataFrame'>
Index: MultiIndex([('2024-01-02', 'AAPL'),
            ('2024-01-03', 'AAPL'),
            ('2024-01-04', 'AAPL'),
            ('2024-01-05', 'AAPL'),
            ('2024-01-08', 'AAPL'),
            ('2024-01-09', 'AAPL'),
            ('2024-01-10', 'AAPL'),
            ('2024-01-11', 'AAPL'),
            ('2024-01-12', 'AAPL'),
            ('2024-01-16', 'AAPL'),
            ...
            ('2024-12-16', 'NVDA'),
            ('2024-12-17', 'NVDA'),
            ('2024-12-18', 'NVDA'),
            ('2024-12-19', 'NVDA'),
            ('2024-12-20', 'NVDA'),
            ('2024-12-23', 'NVDA'),
            ('2024-12-24', 'NVDA'),
            ('2024-12-26', 'NVDA'),
            ('2024-12-27', 'NVDA'),
            ('2024-12-30', 'NVDA')],
           names=['Date', 'Ticker'], length=753)
Columns: Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object', name='Price')

First few rows:
Price                 

In [10]:
# Debug the sector_volatility function step by step
from src.volatility import sector_volatility

# Test just the transformation part
hist_vol = historical_volatility(ohlcv_data, window=20)
print("hist_vol type:", type(hist_vol))
print("hist_vol index type:", type(hist_vol.index))

# Get the latest value for each ticker
if isinstance(hist_vol.index, pd.MultiIndex):
    ticker_vol = hist_vol.groupby(level=1).last().dropna()
    print("\nticker_vol after groupby:")
    print(ticker_vol)
    print("ticker_vol index:", ticker_vol.index)
    
    # Convert to DataFrame
    vol_df = ticker_vol.to_frame('Volatility')
    vol_df.index.name = 'Ticker'
    vol_df = vol_df.reset_index()
    print("\nvol_df after conversion:")
    print(vol_df)
    
    print("\nsector_map structure:")
    print(sector_map)
    
    # Try the merge
    try:
        merged_df = vol_df.merge(sector_map, on='Ticker', how='left')
        print("\nMerge successful!")
        print(merged_df)
    except Exception as e:
        print(f"\nMerge failed: {e}")
        print(f"vol_df columns: {vol_df.columns}")
        print(f"sector_map columns: {sector_map.columns}")
        print(f"vol_df column levels: {vol_df.columns.nlevels}")
        print(f"sector_map column levels: {sector_map.columns.nlevels}")

hist_vol type: <class 'pandas.core.series.Series'>
hist_vol index type: <class 'pandas.core.indexes.multi.MultiIndex'>

ticker_vol after groupby:
Ticker
AAPL    0.162911
MSFT    0.201190
NVDA    0.331227
Name: Adj Close, dtype: float64
ticker_vol index: Index(['AAPL', 'MSFT', 'NVDA'], dtype='object', name='Ticker')

vol_df after conversion:
  Ticker  Volatility
0   AAPL    0.162911
1   MSFT    0.201190
2   NVDA    0.331227

sector_map structure:
  Ticker      Sector
0   AAPL  Technology
1   MSFT  Technology
2   NVDA  Technology

Merge successful!
  Ticker  Volatility      Sector
0   AAPL    0.162911  Technology
1   MSFT    0.201190  Technology
2   NVDA    0.331227  Technology


In [11]:
from src import volatility

In [12]:
heatmap1 = volatility.prepare_time_volatility_heatmap(ohlcv_data, window=30, freq='M')

C:\Users\KIIT\Desktop\Everything Code\Quant\RiskLens\src\volatility.py:152: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vol_pivot = vol_pivot.resample(freq).last()


In [13]:
# Import the updated volatility module
from src import volatility
import importlib
importlib.reload(volatility)

<module 'src.volatility' from 'C:\\Users\\KIIT\\Desktop\\Everything Code\\Quant\\RiskLens\\src\\volatility.py'>

In [14]:
# Test 1: Time-based volatility heatmap (X: Time, Y: Tickers)
print("=== Time-based Volatility Heatmap ===")
time_heatmap = volatility.prepare_time_volatility_heatmap(ohlcv_data, window=30, freq='M')
print(f"Shape: {time_heatmap.shape}")
print(f"Index (Time): {time_heatmap.index}")
print(f"Columns (Tickers): {time_heatmap.columns}")
print("\nSample data:")
print(time_heatmap.head())

=== Time-based Volatility Heatmap ===
Shape: (12, 3)
Index (Time): DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31', '2024-08-31',
               '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31'],
              dtype='datetime64[ns]', name='Date', freq='ME')
Columns (Tickers): Index(['AAPL', 'MSFT', 'NVDA'], dtype='object', name='Ticker')

Sample data:
Ticker          AAPL      MSFT      NVDA
Date                                    
2024-01-31       NaN  1.334960  2.606513
2024-02-29  0.174913  0.194605  0.575815
2024-03-31  0.212968  0.195070  0.640992
2024-04-30  0.254091  0.198502  0.511645
2024-05-31  0.236263  0.220602  0.600486


C:\Users\KIIT\Desktop\Everything Code\Quant\RiskLens\src\volatility.py:152: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vol_pivot = vol_pivot.resample(freq).last()


In [15]:
# Test 2: Cross-sectional volatility comparison (X: Vol Measures, Y: Tickers)
print("=== Cross-sectional Volatility Heatmap ===")
cross_sectional_heatmap = volatility.prepare_cross_sectional_heatmap(ohlcv_data, benchmark_df, window=30)
print(f"Shape: {cross_sectional_heatmap.shape}")
print(f"Index (Tickers): {cross_sectional_heatmap.index}")
print(f"Columns (Vol Measures): {cross_sectional_heatmap.columns}")
print("\nData:")
print(cross_sectional_heatmap)

=== Cross-sectional Volatility Heatmap ===
Shape: (3, 4)
Index (Tickers): Index(['AAPL', 'MSFT', 'NVDA'], dtype='object')
Columns (Vol Measures): Index(['Historical Vol', 'Parkinson Vol', 'Garman-Klass Vol', 'Beta-Adj Vol'], dtype='object')

Data:
      Historical Vol  Parkinson Vol  Garman-Klass Vol  Beta-Adj Vol
AAPL        0.151543       1.182974          0.122430      0.127288
MSFT        0.200878       1.251927          0.146575      0.155642
NVDA        0.364521       1.816383          0.473937      0.294483
Shape: (3, 4)
Index (Tickers): Index(['AAPL', 'MSFT', 'NVDA'], dtype='object')
Columns (Vol Measures): Index(['Historical Vol', 'Parkinson Vol', 'Garman-Klass Vol', 'Beta-Adj Vol'], dtype='object')

Data:
      Historical Vol  Parkinson Vol  Garman-Klass Vol  Beta-Adj Vol
AAPL        0.151543       1.182974          0.122430      0.127288
MSFT        0.200878       1.251927          0.146575      0.155642
NVDA        0.364521       1.816383          0.473937      0.294483


In [16]:
# Test 3: Sector volatility heatmap (X: Vol Measures, Y: Sectors)
print("=== Sector Volatility Heatmap ===")
sector_heatmap = volatility.prepare_sector_volatility_heatmap(ohlcv_data, sector_map, window=30)
print(f"Shape: {sector_heatmap.shape}")
print(f"Index (Sectors): {sector_heatmap.index}")
print(f"Columns (Vol Measures): {sector_heatmap.columns}")
print("\nData:")
print(sector_heatmap)

=== Sector Volatility Heatmap ===
Shape: (1, 3)
Index (Sectors): Index(['Technology'], dtype='object')
Columns (Vol Measures): Index(['Historical Vol', 'Parkinson Vol', 'Garman-Klass Vol'], dtype='object')

Data:
            Historical Vol  Parkinson Vol  Garman-Klass Vol
Technology        0.238981       1.417095          0.247647


In [17]:
# Test 4: Volatility correlation heatmap (X: Tickers, Y: Tickers)
print("=== Volatility Correlation Heatmap ===")
correlation_heatmap = volatility.prepare_correlation_heatmap(ohlcv_data, window=30)
print(f"Shape: {correlation_heatmap.shape}")
print(f"Index (Tickers): {correlation_heatmap.index}")
print(f"Columns (Tickers): {correlation_heatmap.columns}")
print("\nCorrelation Matrix:")
print(correlation_heatmap)

=== Volatility Correlation Heatmap ===
Shape: (3, 3)
Index (Tickers): Index(['AAPL', 'MSFT', 'NVDA'], dtype='object')
Columns (Tickers): Index(['AAPL', 'MSFT', 'NVDA'], dtype='object')

Correlation Matrix:
          AAPL      MSFT      NVDA
AAPL  1.000000 -0.277102  0.284486
MSFT -0.277102  1.000000  0.100301
NVDA  0.284486  0.100301  1.000000


## Heatmap Visualization Examples

Now let's create actual heatmap visualizations using plotly or seaborn.

In [18]:
import plotly.graph_objects as go
import plotly.express as px

# Create time-based volatility heatmap
fig = go.Figure(data=go.Heatmap(
    z=time_heatmap.values,
    x=time_heatmap.columns,  # Tickers
    y=time_heatmap.index.strftime('%Y-%m'),  # Time periods
    colorscale='RdYlBu_r',
    hoverongaps=False
))

fig.update_layout(
    title='Time-based Volatility Heatmap',
    xaxis_title='Tickers',
    yaxis_title='Time Periods (Monthly)',
    width=800,
    height=500
)

fig.show()

In [19]:
# Create cross-sectional volatility comparison heatmap
fig = go.Figure(data=go.Heatmap(
    z=cross_sectional_heatmap.values,
    x=cross_sectional_heatmap.columns,  # Volatility measures
    y=cross_sectional_heatmap.index,    # Tickers
    colorscale='Viridis',
    text=cross_sectional_heatmap.values.round(4),
    texttemplate='%{text}',
    textfont={"size": 10}
))

fig.update_layout(
    title='Cross-sectional Volatility Comparison',
    xaxis_title='Volatility Measures',
    yaxis_title='Tickers',
    width=800,
    height=400
)

fig.show()

In [20]:
# Debug the benchmark data alignment issue
print("=== Debugging Benchmark Data Alignment ===")
print(f"ohlcv_data shape: {ohlcv_data.shape}")
print(f"benchmark_df shape: {benchmark_df.shape}")
print(f"ohlcv_data index: {ohlcv_data.index}")
print(f"benchmark_df index: {benchmark_df.index}")

# Check if benchmark has the same structure
print(f"\nbenchmark_df columns: {benchmark_df.columns}")
print(f"benchmark_df head:\n{benchmark_df.head()}")

# Test returns calculation for one ticker
test_ticker = 'AAPL'
ticker_data = ohlcv_data.xs(test_ticker, level=1)
print(f"\nTicker data for {test_ticker}:")
print(f"Shape: {ticker_data.shape}")
print(f"Columns: {ticker_data.columns}")

# Test returns alignment
from src.volatility import compute_daily_returns
ticker_returns = compute_daily_returns(ticker_data).dropna()
bench_returns = compute_daily_returns(benchmark_df).dropna()

print(f"\nTicker returns shape: {ticker_returns.shape}")
print(f"Benchmark returns shape: {bench_returns.shape}")
print(f"Ticker returns index range: {ticker_returns.index.min()} to {ticker_returns.index.max()}")
print(f"Benchmark returns index range: {bench_returns.index.min()} to {bench_returns.index.max()}")

# Test alignment
aligned_ticker, aligned_bench = ticker_returns.align(bench_returns, join='inner')
print(f"\nAfter alignment:")
print(f"Aligned ticker shape: {aligned_ticker.shape}")
print(f"Aligned benchmark shape: {aligned_bench.shape}")

=== Debugging Benchmark Data Alignment ===
ohlcv_data shape: (753, 6)
benchmark_df shape: (251, 7)
ohlcv_data index: MultiIndex([('2024-01-02', 'AAPL'),
            ('2024-01-03', 'AAPL'),
            ('2024-01-04', 'AAPL'),
            ('2024-01-05', 'AAPL'),
            ('2024-01-08', 'AAPL'),
            ('2024-01-09', 'AAPL'),
            ('2024-01-10', 'AAPL'),
            ('2024-01-11', 'AAPL'),
            ('2024-01-12', 'AAPL'),
            ('2024-01-16', 'AAPL'),
            ...
            ('2024-12-16', 'NVDA'),
            ('2024-12-17', 'NVDA'),
            ('2024-12-18', 'NVDA'),
            ('2024-12-19', 'NVDA'),
            ('2024-12-20', 'NVDA'),
            ('2024-12-23', 'NVDA'),
            ('2024-12-24', 'NVDA'),
            ('2024-12-26', 'NVDA'),
            ('2024-12-27', 'NVDA'),
            ('2024-12-30', 'NVDA')],
           names=['Date', 'Ticker'], length=753)
benchmark_df index: DatetimeIndex(['2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05',
      

In [21]:
# Test 2: Cross-sectional volatility comparison (X: Vol Measures, Y: Tickers) - FIXED
print("=== Cross-sectional Volatility Heatmap (Fixed) ===")
try:
    cross_sectional_heatmap = volatility.prepare_cross_sectional_heatmap(ohlcv_data, benchmark_df, window=30)
    print(f"Shape: {cross_sectional_heatmap.shape}")
    print(f"Index (Tickers): {cross_sectional_heatmap.index}")
    print(f"Columns (Vol Measures): {cross_sectional_heatmap.columns}")
    print("\nData:")
    print(cross_sectional_heatmap)
except Exception as e:
    print(f"Error: {e}")
    print("Trying without benchmark...")
    cross_sectional_heatmap = volatility.prepare_cross_sectional_heatmap(ohlcv_data, None, window=30)
    print(f"Shape: {cross_sectional_heatmap.shape}")
    print(f"Data:\n{cross_sectional_heatmap}")

=== Cross-sectional Volatility Heatmap (Fixed) ===
Shape: (3, 4)
Index (Tickers): Index(['AAPL', 'MSFT', 'NVDA'], dtype='object')
Columns (Vol Measures): Index(['Historical Vol', 'Parkinson Vol', 'Garman-Klass Vol', 'Beta-Adj Vol'], dtype='object')

Data:
      Historical Vol  Parkinson Vol  Garman-Klass Vol  Beta-Adj Vol
AAPL        0.151543       1.182974          0.122430      0.127288
MSFT        0.200878       1.251927          0.146575      0.155642
NVDA        0.364521       1.816383          0.473937      0.294483


## Complete Volatility Heatmap Visualization Suite

This section demonstrates all 4 types of volatility heatmaps with proper axis configurations.

In [22]:
# Reload the volatility module and prepare all heatmap data
import importlib
importlib.reload(volatility)

print("Preparing all heatmap datasets...")

# 1. Time-based heatmap data
time_heatmap = volatility.prepare_time_volatility_heatmap(ohlcv_data, window=30, freq='M')
print(f"✓ Time heatmap: {time_heatmap.shape}")

# 2. Cross-sectional heatmap data
cross_sectional_heatmap = volatility.prepare_cross_sectional_heatmap(ohlcv_data, benchmark_df, window=30)
print(f"✓ Cross-sectional heatmap: {cross_sectional_heatmap.shape}")

# 3. Sector heatmap data
sector_heatmap = volatility.prepare_sector_volatility_heatmap(ohlcv_data, sector_map, window=30)
print(f"✓ Sector heatmap: {sector_heatmap.shape}")

# 4. Correlation heatmap data
correlation_heatmap = volatility.prepare_correlation_heatmap(ohlcv_data, window=30)
print(f"✓ Correlation heatmap: {correlation_heatmap.shape}")

print("\nAll heatmap data prepared successfully!")

Preparing all heatmap datasets...
✓ Time heatmap: (12, 3)
✓ Cross-sectional heatmap: (3, 4)
✓ Sector heatmap: (1, 3)
✓ Correlation heatmap: (3, 3)

All heatmap data prepared successfully!


C:\Users\KIIT\Desktop\Everything Code\Quant\RiskLens\src\volatility.py:152: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [23]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

# 1. TIME-BASED VOLATILITY HEATMAP
# X-axis: Time periods (Monthly), Y-axis: Tickers
fig1 = go.Figure(data=go.Heatmap(
    z=time_heatmap.T.values,  # Transpose to get tickers on Y-axis
    x=time_heatmap.index.strftime('%Y-%m'),  # Time periods on X-axis
    y=time_heatmap.columns,  # Tickers on Y-axis
    colorscale='RdYlBu_r',
    colorbar=dict(title="Volatility"),
    hoverongaps=False,
    hovertemplate='<b>%{y}</b><br>Period: %{x}<br>Volatility: %{z:.4f}<extra></extra>'
))

fig1.update_layout(
    title={
        'text': 'Time-based Volatility Heatmap<br><sub>X-axis: Time Periods (Monthly) | Y-axis: Tickers</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Time Periods (Monthly)',
    yaxis_title='Tickers',
    width=1000,
    height=400,
    font=dict(size=12)
)

fig1.show()

In [24]:
# 2. CROSS-SECTIONAL VOLATILITY COMPARISON HEATMAP
# X-axis: Volatility Measures, Y-axis: Tickers
fig2 = go.Figure(data=go.Heatmap(
    z=cross_sectional_heatmap.values,
    x=cross_sectional_heatmap.columns,  # Volatility measures on X-axis
    y=cross_sectional_heatmap.index,    # Tickers on Y-axis
    colorscale='Viridis',
    colorbar=dict(title="Volatility Value"),
    text=cross_sectional_heatmap.values.round(4),
    texttemplate='%{text}',
    textfont={"size": 10, "color": "white"},
    hovertemplate='<b>%{y}</b><br>Measure: %{x}<br>Value: %{z:.4f}<extra></extra>'
))

fig2.update_layout(
    title={
        'text': 'Cross-sectional Volatility Comparison<br><sub>X-axis: Volatility Measures | Y-axis: Tickers</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Volatility Measures',
    yaxis_title='Tickers',
    width=900,
    height=500,
    font=dict(size=12)
)

fig2.show()

In [25]:
# 3. SECTOR VOLATILITY HEATMAP
# X-axis: Volatility Measures, Y-axis: Sectors
fig3 = go.Figure(data=go.Heatmap(
    z=sector_heatmap.values,
    x=sector_heatmap.columns,  # Volatility measures on X-axis
    y=sector_heatmap.index,    # Sectors on Y-axis
    colorscale='Plasma',
    colorbar=dict(title="Average Volatility"),
    text=sector_heatmap.values.round(4),
    texttemplate='%{text}',
    textfont={"size": 12, "color": "white"},
    hovertemplate='<b>%{y}</b><br>Measure: %{x}<br>Avg Volatility: %{z:.4f}<extra></extra>'
))

fig3.update_layout(
    title={
        'text': 'Sector Average Volatility Heatmap<br><sub>X-axis: Volatility Measures | Y-axis: Sectors</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Volatility Measures',
    yaxis_title='Sectors',
    width=800,
    height=400,
    font=dict(size=12)
)

fig3.show()

In [26]:
# 4. VOLATILITY CORRELATION HEATMAP
# X-axis: Tickers, Y-axis: Tickers
fig4 = go.Figure(data=go.Heatmap(
    z=correlation_heatmap.values,
    x=correlation_heatmap.columns,  # Tickers on X-axis
    y=correlation_heatmap.index,    # Tickers on Y-axis
    colorscale='RdBu',
    zmid=0,  # Center the colorscale at 0
    colorbar=dict(title="Correlation"),
    text=correlation_heatmap.values.round(3),
    texttemplate='%{text}',
    textfont={"size": 11, "color": "black"},
    hovertemplate='<b>%{y} vs %{x}</b><br>Correlation: %{z:.3f}<extra></extra>'
))

fig4.update_layout(
    title={
        'text': 'Volatility Correlation Matrix<br><sub>X-axis: Tickers | Y-axis: Tickers</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Tickers',
    yaxis_title='Tickers',
    width=600,
    height=600,
    font=dict(size=12)
)

fig4.show()

## Combined Dashboard View

Create a comprehensive dashboard showing all heatmaps in a single view.

In [27]:
# COMBINED DASHBOARD - All 4 heatmaps in subplots
fig_combined = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Time-based Volatility (X: Time, Y: Tickers)',
        'Cross-sectional Comparison (X: Measures, Y: Tickers)',
        'Sector Analysis (X: Measures, Y: Sectors)', 
        'Volatility Correlations (X: Tickers, Y: Tickers)'
    ],
    specs=[[{"type": "heatmap"}, {"type": "heatmap"}],
           [{"type": "heatmap"}, {"type": "heatmap"}]]
)

# Add heatmap 1: Time-based
fig_combined.add_trace(
    go.Heatmap(
        z=time_heatmap.T.values,
        x=time_heatmap.index.strftime('%Y-%m'),
        y=time_heatmap.columns,
        colorscale='RdYlBu_r',
        showscale=False,
        hovertemplate='Period: %{x}<br>Ticker: %{y}<br>Vol: %{z:.4f}<extra></extra>'
    ),
    row=1, col=1
)

# Add heatmap 2: Cross-sectional
fig_combined.add_trace(
    go.Heatmap(
        z=cross_sectional_heatmap.values,
        x=cross_sectional_heatmap.columns,
        y=cross_sectional_heatmap.index,
        colorscale='Viridis',
        showscale=False,
        hovertemplate='Measure: %{x}<br>Ticker: %{y}<br>Value: %{z:.4f}<extra></extra>'
    ),
    row=1, col=2
)

# Add heatmap 3: Sector
fig_combined.add_trace(
    go.Heatmap(
        z=sector_heatmap.values,
        x=sector_heatmap.columns,
        y=sector_heatmap.index,
        colorscale='Plasma',
        showscale=False,
        hovertemplate='Measure: %{x}<br>Sector: %{y}<br>Avg Vol: %{z:.4f}<extra></extra>'
    ),
    row=2, col=1
)

# Add heatmap 4: Correlation
fig_combined.add_trace(
    go.Heatmap(
        z=correlation_heatmap.values,
        x=correlation_heatmap.columns,
        y=correlation_heatmap.index,
        colorscale='RdBu',
        zmid=0,
        showscale=False,
        hovertemplate='%{y} vs %{x}<br>Correlation: %{z:.3f}<extra></extra>'
    ),
    row=2, col=2
)

fig_combined.update_layout(
    title_text="Volatility Analysis Dashboard - All Heatmap Types",
    title_x=0.5,
    width=1400,
    height=800,
    font=dict(size=10)
)

fig_combined.show()

## Summary of Heatmap Axis Configurations

### 1. Time-based Volatility Heatmap
- **X-axis**: Time Periods (Daily/Weekly/Monthly/Quarterly)
- **Y-axis**: Tickers/Assets
- **Values**: Historical volatility over time
- **Use Case**: Track volatility evolution across time

### 2. Cross-sectional Volatility Comparison
- **X-axis**: Different Volatility Measures (Historical, Parkinson, Garman-Klass, Beta-Adjusted)
- **Y-axis**: Tickers/Assets  
- **Values**: Latest volatility values for each measure
- **Use Case**: Compare different volatility calculation methods

### 3. Sector Volatility Analysis
- **X-axis**: Volatility Measures
- **Y-axis**: Sectors
- **Values**: Average volatility by sector
- **Use Case**: Sector-level risk assessment

### 4. Volatility Correlation Matrix
- **X-axis**: Tickers/Assets
- **Y-axis**: Tickers/Assets
- **Values**: Correlation coefficients between volatilities
- **Use Case**: Portfolio diversification analysis

In [28]:
# Export heatmap data for further analysis
print("Exporting heatmap data...")

# Save all heatmap data to Excel with multiple sheets
with pd.ExcelWriter('volatility_heatmaps_analysis.xlsx') as writer:
    time_heatmap.to_excel(writer, sheet_name='Time_Based')
    cross_sectional_heatmap.to_excel(writer, sheet_name='Cross_Sectional')
    sector_heatmap.to_excel(writer, sheet_name='Sector_Analysis')
    correlation_heatmap.to_excel(writer, sheet_name='Correlations')

print("✓ All heatmap data exported to 'volatility_heatmaps_analysis.xlsx'")

# Display summary statistics
print("\n=== HEATMAP SUMMARY STATISTICS ===")
print(f"Time-based heatmap shape: {time_heatmap.shape}")
print(f"Cross-sectional heatmap shape: {cross_sectional_heatmap.shape}")
print(f"Sector heatmap shape: {sector_heatmap.shape}")
print(f"Correlation heatmap shape: {correlation_heatmap.shape}")

print(f"\nVolatility ranges:")
print(f"Historical Vol: {cross_sectional_heatmap['Historical Vol'].min():.4f} - {cross_sectional_heatmap['Historical Vol'].max():.4f}")
print(f"Parkinson Vol: {cross_sectional_heatmap['Parkinson Vol'].min():.4f} - {cross_sectional_heatmap['Parkinson Vol'].max():.4f}")
print(f"Garman-Klass Vol: {cross_sectional_heatmap['Garman-Klass Vol'].min():.4f} - {cross_sectional_heatmap['Garman-Klass Vol'].max():.4f}")

Exporting heatmap data...
✓ All heatmap data exported to 'volatility_heatmaps_analysis.xlsx'

=== HEATMAP SUMMARY STATISTICS ===
Time-based heatmap shape: (12, 3)
Cross-sectional heatmap shape: (3, 4)
Sector heatmap shape: (1, 3)
Correlation heatmap shape: (3, 3)

Volatility ranges:
Historical Vol: 0.1515 - 0.3645
Parkinson Vol: 1.1830 - 1.8164
Garman-Klass Vol: 0.1224 - 0.4739


In [29]:
# Debug benchmark data structure to fix beta calculation
print("=== Benchmark Data Structure Analysis ===")
print(f"benchmark_df type: {type(benchmark_df)}")
print(f"benchmark_df shape: {benchmark_df.shape}")
print(f"benchmark_df index: {benchmark_df.index}")
print(f"benchmark_df columns: {benchmark_df.columns}")
print(f"benchmark_df index has levels: {hasattr(benchmark_df.index, 'levels')}")
print("\nFirst few rows:")
print(benchmark_df.head())

# Test the fix by reloading and running
importlib.reload(volatility)

print("\n=== Testing Fixed Cross-sectional Heatmap ===")
cross_sectional_heatmap_fixed = volatility.prepare_cross_sectional_heatmap(ohlcv_data, benchmark_df, window=30)
print(f"✓ Fixed cross-sectional heatmap: {cross_sectional_heatmap_fixed.shape}")
print("\nFixed data:")
print(cross_sectional_heatmap_fixed)

=== Benchmark Data Structure Analysis ===
benchmark_df type: <class 'pandas.core.frame.DataFrame'>
benchmark_df shape: (251, 7)
benchmark_df index: DatetimeIndex(['2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05',
               '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11',
               '2024-01-12', '2024-01-16',
               ...
               '2024-12-16', '2024-12-17', '2024-12-18', '2024-12-19',
               '2024-12-20', '2024-12-23', '2024-12-24', '2024-12-26',
               '2024-12-27', '2024-12-30'],
              dtype='datetime64[ns]', name='Date', length=251, freq=None)
benchmark_df columns: MultiIndex([('Adj Close', '^GSPC'),
            (    'Close', '^GSPC'),
            (     'High', '^GSPC'),
            (      'Low', '^GSPC'),
            (     'Open', '^GSPC'),
            (   'Volume', '^GSPC'),
            (  'Returns',      '')],
           names=['Price', 'Ticker'])
benchmark_df index has levels: False

First few rows:
Price         Adj Clo

In [30]:
# Update the visualization with the fixed data
cross_sectional_heatmap = cross_sectional_heatmap_fixed

print("=== Updated Heatmap Data (No Warnings) ===")
print(f"Time heatmap: {time_heatmap.shape}")
print(f"Cross-sectional heatmap: {cross_sectional_heatmap.shape}")
print(f"Sector heatmap: {sector_heatmap.shape}")
print(f"Correlation heatmap: {correlation_heatmap.shape}")
print("\nAll heatmap data prepared successfully without warnings!")

=== Updated Heatmap Data (No Warnings) ===
Time heatmap: (12, 3)
Cross-sectional heatmap: (3, 4)
Sector heatmap: (1, 3)
Correlation heatmap: (3, 3)

All heatmap data prepared successfully without warnings!
